<div style="
    display: flex;
    justify-content: center;
    align-items: center;
    background: linear-gradient(to bottom, #F5F5DC, #C0C0C0);
    padding: 10px;
    border-radius: 10px;
">
    <img src="../samsung.png" alt="Samsung Innovation Campus" style="border-radius: 5px;">
</div>
<div style="
    text-align: center;
    font-style: italic;
">
    Este proyecto fue desarrollado dentro del marco del programa Samsun Innovation Campus 2024
</div>

<b>PRESENTAN</b>: José Armando Ramírez Islas & Jorge Octavio Nicolás Díaz

# __Preprocesamiento__

Las características con las que cuenta el dataset son:

| Característica     | Descripción                                                                 |
|--------------|-----------------------------------------------------------------------------|
| dur      | Duración de la conexión en segundos.                                         |
| proto    | Protocolo de comunicación utilizado (ej. TCP, UDP, ICMP).                   |
| dir      | Dirección del flujo de tráfico (ej. → si es de origen a destino, o ← si es de destino a origen). |
| state    | Estado de la conexión (ej. CON para conexiones establecidas, INT para interrumpidas). |
| stos / dtos | Tipo de servicio (ToS) del tráfico enviado y recibido. Son valores que indican la prioridad del paquete en la red. |
| tot_pkts | Número total de paquetes enviados en la conexión.                           |
| tot_bytes| Número total de bytes transferidos.                                         |
| src_bytes| Cantidad de bytes enviados desde la IP de origen.                            |
| label    | Etiqueta que indica si el tráfico es normal o pertenece a una botnet (tráfico malicioso). |
| Family   | Especie de botnet detectada (ej. Neris, Rbot, Virut, Murlo, etc.).           |

### 1️⃣ __Importacion de modulos__

In [1]:
import dask.dataframe as dd
import os

### 2️⃣ __Leer el CSV forzando las columnas conflictivas a 'object' (string)__

In [2]:
ctu13_df = dd.read_csv('../data/total.csv', dtype={"Dport": "object", "Sport": "object"}, assume_missing=True)

### 3️⃣ __Verificar los tipos de datos antes de procesar__

In [3]:
print(ctu13_df.dtypes)

StartTime      string[pyarrow]
Dur                    float64
Proto          string[pyarrow]
SrcAddr        string[pyarrow]
Sport          string[pyarrow]
Dir            string[pyarrow]
DstAddr        string[pyarrow]
Dport          string[pyarrow]
State          string[pyarrow]
sTos                   float64
dTos                   float64
TotPkts                float64
TotBytes               float64
SrcBytes               float64
Label          string[pyarrow]
BOTNET_NAME    string[pyarrow]
dtype: object


### 4️⃣ __Función para convertir hex a int y evitar errores__

In [4]:
def safe_hex_to_int(value):
    try:
        if isinstance(value, str) and value.startswith("0x"):  
            return int(value, 16)
        return int(value)  
    except ValueError:
        return None 

### 5️⃣ __Aplicar conversión a las columnas__

In [5]:
ctu13_df["Dport"] = ctu13_df["Dport"].map(safe_hex_to_int, meta=("Dport", "float64"))
ctu13_df["Sport"] = ctu13_df["Sport"].map(safe_hex_to_int, meta=("Sport", "float64"))

In [6]:
ctu13_df.head()

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label,BOTNET_NAME
0,2011/08/16 14:18:55.889839,3599.725830,tcp,88.176.79.163,49375,<?>,147.32.84.172,46696,A_PA,0.0,0.0,274708.0,271837036.0,6887036.0,flow=Background,Murlo
1,2011/08/16 14:18:55.890497,3532.512939,tcp,134.2.99.108,12106,<?>,147.32.84.59,51472,PA_PA,0.0,0.0,513.0,42640.0,21205.0,flow=Background-Established-cmpgw-CVUT,Murlo
2,2011/08/16 14:18:55.892530,3599.922852,udp,109.80.124.147,10227,<->,147.32.86.77,43332,CON,0.0,0.0,69490.0,14362350.0,11870607.0,flow=Background-UDP-Established,Murlo
3,2011/08/16 14:18:55.892624,82.457962,tcp,147.32.84.59,34394,<?>,74.125.39.117,80,FPA_FPA,0.0,0.0,12526.0,8472421.0,8118714.0,flow=Background-Established-cmpgw-CVUT,Murlo
4,2011/08/16 14:18:55.894121,3599.954346,tcp,88.176.79.163,49410,<?>,147.32.84.172,52935,A_PA,0.0,0.0,285606.0,281932488.0,7230814.0,flow=Background,Murlo


### 6️⃣ __¿Que columnas tienen valores nulos?__

In [7]:
ctu13_df.isnull().sum().compute()

StartTime           0
Dur                 0
Proto               0
SrcAddr             0
Sport           72675
Dir                 0
DstAddr             0
Dport          133291
State            1235
sTos            82758
dTos           643122
TotPkts             0
TotBytes            0
SrcBytes            0
Label               0
BOTNET_NAME         0
dtype: int64

### 7️⃣ __Rellenando los registros donde hay valores nulos__

In [8]:
ctu13_df['State'].value_counts().nlargest(1).compute()

State
CON    6208714
Name: count, dtype: int64

In [9]:
ctu13_df['State'] = ctu13_df.State.fillna(value='CON')

In [10]:
ctu13_df['sTos'].describe().compute()

count    8.323752e+06
mean     6.802930e-02
std      3.567048e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.920000e+02
Name: sTos, dtype: float64

In [11]:
ctu13_df['dTos'].describe().compute()

count    7.763388e+06
mean     4.289364e-04
std      3.358086e-02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.000000e+00
Name: dTos, dtype: float64

In [12]:
ctu13_df['sTos'] = ctu13_df.sTos.fillna(value=0.0)
ctu13_df['dTos'] = ctu13_df.dTos.fillna(value=0.0)

In [13]:
ctu13_df['Sport'] = ctu13_df['Sport'].ffill().bfill()
ctu13_df['Dport'] = ctu13_df['Dport'].ffill().bfill()

### 8️⃣ __Validando si pertenece a una botnet__

In [14]:
def convert_label(sample):
    if isinstance(sample, str) and "Botnet" in sample: 
        return 1
    else: 
        return 0

### 9️⃣ __Agregamos una columna que identifica si pertenece a una botnet o no__

In [15]:
ctu13_df['is_botnet'] = ctu13_df['Label'].apply(convert_label, meta=('Label', 'int64'))

In [16]:
ctu13_df[ctu13_df['is_botnet'] == 1].sample(frac=0.1, random_state=45).compute()

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label,BOTNET_NAME,is_botnet
291231,2011/08/16 15:17:08.835592,18.202251,udp,147.32.84.165,29096.0,<->,183.60.16.10,8000.0,CON,0.0,0.0,23.0,8534.0,2904.0,flow=From-Botnet-V49-UDP-Established-Custom-En...,Murlo,1
162509,2011/08/16 14:49:18.902462,65.532639,udp,147.32.84.165,21957.0,<->,183.60.16.244,8000.0,CON,0.0,0.0,97.0,40218.0,11036.0,flow=From-Botnet-V49-UDP-Established-Custom-En...,Murlo,1
347153,2011/08/16 15:28:25.140271,0.000000,tcp,147.32.84.165,1856.0,->,147.32.80.107,135.0,S_,0.0,0.0,1.0,62.0,62.0,flow=From-Botnet-V49-TCP-Attempt,Murlo,1
337676,2011/08/16 15:26:18.138990,61.281811,tcp,147.32.84.165,4493.0,->,222.189.228.111,3389.0,FSPA_FSPA,0.0,0.0,10.0,1076.0,437.0,flow=From-Botnet-V49-TCP-CC74-HTTP-Custom-Port...,Murlo,1
358515,2011/08/16 15:30:49.348106,0.000000,tcp,147.32.84.165,3443.0,->,147.32.83.64,135.0,S_,0.0,0.0,1.0,62.0,62.0,flow=From-Botnet-V49-TCP-Attempt,Murlo,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439145,2011/08/16 09:34:51.024297,1.207547,tcp,147.32.84.165,23050.0,->,184.173.217.40,443.0,S_RA,0.0,0.0,6.0,366.0,186.0,flow=From-Botnet-V54-TCP-Attempt,Virut,1
86918,2011/08/16 06:39:52.666626,1.308002,tcp,147.32.84.165,17721.0,->,184.173.217.40,443.0,S_RA,0.0,0.0,6.0,366.0,186.0,flow=From-Botnet-V54-TCP-Attempt,Virut,1
425378,2011/08/16 09:30:28.514024,1.307745,tcp,147.32.84.165,22909.0,->,184.173.217.40,443.0,S_RA,0.0,0.0,6.0,366.0,186.0,flow=From-Botnet-V54-TCP-Attempt,Virut,1
189914,2011/08/16 07:46:50.135599,1.307472,tcp,147.32.84.165,19782.0,->,184.173.217.40,443.0,S_RA,0.0,0.0,6.0,366.0,186.0,flow=From-Botnet-V54-TCP-Attempt,Virut,1


### 🔟 __Columnas que contienen datos categoricos__

In [17]:
# Categorical feature names
ctu13_df.select_dtypes(exclude='number').columns

Index(['StartTime', 'Proto', 'SrcAddr', 'Dir', 'DstAddr', 'State', 'Label',
       'BOTNET_NAME'],
      dtype='object')

### 1️⃣1️⃣ __Columnas que contienen datos numericos__

In [18]:
# Numeric features names
ctu13_df.select_dtypes(include='number').columns

Index(['Dur', 'Sport', 'Dport', 'sTos', 'dTos', 'TotPkts', 'TotBytes',
       'SrcBytes', 'is_botnet'],
      dtype='object')

### 1️⃣2️⃣ __Cantidad de registros que son botnets y los que no__

In [19]:
print(ctu13_df['is_botnet'].value_counts().compute())

is_botnet
1     338370
0    8068140
Name: count, dtype: int64


### 1️⃣3️⃣ __Filtrar solo los registros donde is_botnet == 1 para contar los tipos de botnets__

In [20]:
# Filtrar solo los registros donde is_botnet == 1 
print(ctu13_df[ctu13_df['is_botnet'] == 1]['BOTNET_NAME'].value_counts().compute())

BOTNET_NAME
Virut     40904
Neris    184987
RBot     106352
Murlo      6127
Name: count, dtype: int64


### 1️⃣4️⃣ __Generar un nuevo dataset preprocesado__

In [21]:
ctu13_df.compute().to_csv(os.path.join('preprocessing_data', 'dataset_procesado.csv'), index=False)